In [ ]:
import numpy as np
import time
import math
T = 4
num_s = 3
num_a = 1
num_e = 2

def policy(st,mu_a_t,mu_s_t,matrix_p_t):
    at = mu_a_t+ np.dot(matrix_p_t, st-mu_s_t)
    return at

def reward(mt,bt,ct,at,st):
    rt = mt + np.dot(bt,at)+ np.dot(ct,st)
    return rt

def f(st,at,mu_s_t1,mu_s_t,mu_a_t,beta_s_t, beta_a_t):
    s_t1 = mu_s_t1 + np.dot(beta_s_t, st-mu_s_t) + np.dot(beta_a_t, at-mu_a_t)
    return s_t1



# 这是r的参数
m_fix = np.ones(T)
m_fix = m_fix / 1
b = 2*np.ones((T, num_a))
#b[2] = 6
c = np.ones((T, num_s))
c = c / 1
#c[2] = 0.5*np.array([1,3,5])
#c[3] = 1*np.array([5,3,1])


# policy的矩阵参数 num_s*num_a
#matrix_p = np.random.normal(0.1, 0, size=(T, num_a, num_s))
matrix_p = 0.1*np.ones((T, num_a, num_s))
matrix_p[2] = 0.2*np.array([1,2,3])
matrix_p[3] = 0.5*np.array([3.5,2,2])



# 抽样所需要的参数，首先要抽样mu的标准差，假定从一个均匀分布中抽取



# 要估计真实的，首先要计算Rtt矩阵，以及Rt
'''
e2 e3 两个factor

'''
H = 3
def R(tt,t):
    R = np.eye(num_s)
    while tt <= t:
        tmp = beta_s[tt] + np.dot(beta_a[tt], matrix_p[tt])
        R = np.dot(R,tmp)
        tt = tt+1
    return R

alpha = np.zeros((T,num_s))
for t in range(T):
    alpha[t] = np.dot(b[t],matrix_p[t]) + c[t]

def g(subset):
    neg_list = [i for i in range(6)]
    u_list = sorted(subset) # 0-5组成的子列表
    for i in u_list:
        neg_list.remove(i)
    R_neg_u = np.array([])
    R_u = np.array([])
    for i in range(6):
        if i in u_list:
            R_u=np.append(R_u,RR[i+3])
        else:
            R_neg_u = np.append(R_neg_u,RR[i+3])
    sigma_neg_u = np.zeros((len(neg_list),len(u_list)))
    for i in range(len(neg_list)):
        for j in range(len(u_list)):
            if neg_list[i]<=2 and u_list[j]<=2:
                sigma_neg_u[i][j] = cov_1[neg_list[i]][u_list[j]]
            if neg_list[i]>=3 and u_list[j] >=3:
                sigma_neg_u[i][j] = cov_2[neg_list[i]-3][u_list[j]-3]
    sigma_u = np.zeros((len(u_list),len(u_list)))
    for i in range(len(u_list)):
        for j in range(len(u_list)):
            if u_list[i]<=2 and u_list[j]<=2:
                sigma_u[i][j] = cov_1[u_list[i]][u_list[j]]
            if u_list[i]>=3 and u_list[j] >=3:
                sigma_u[i][j] = cov_2[u_list[i]-3][u_list[j]-3]
    res = (R_neg_u @ sigma_neg_u @ np.linalg.inv(sigma_u) + R_u)@sigma_u@(R_neg_u @ sigma_neg_u @ np.linalg.inv(sigma_u) + R_u)

    return res

# 生成子集
def generate_subsets(nums):
    def backtrack(start, path):
        subsets.append(path[:])
        for i in range(start, len(nums)):
            path.append(nums[i])
            backtrack(i + 1, path)
            path.pop()

    subsets = []
    backtrack(0, [])
    return subsets
# 阶乘函数
def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

def shapley(i,j):
    e_list = [h for h in range(6)]
    e_list.remove(i)
    e_list.remove(j)
    all_subset = generate_subsets(e_list)
    Sh = 0
    for u in all_subset:
        tmp = factorial(6-len(u)-2)*factorial(len(u))/factorial(5)
        Sh = Sh +tmp*(g(u+[i,j])-g(u+[i])-g(u+[j])+g(u))
    return Sh

def Sh_single(i):
    e_list = [h for h in range(6)]
    e_list.remove(i)
    all_subset = generate_subsets(e_list)
    Sh = 0
    for u in all_subset:
        tmp = factorial(6-len(u)-1)*factorial(len(u))/factorial(6)
        Sh = Sh +tmp*(g(u+[i])-g(u))
    return Sh


# 接下来物品希望开始计算，e1和e1的协方差矩阵，e1和e2的协方差矩阵，e2和e2的协方差矩阵
'''
cov1_1 = np.zeros((num_s,num_s))
cov1_2 = np.zeros((num_s,num_s))
cov2_2 = np.zeros((num_s,num_s))
'''
Shapley_value = np.zeros((6,6))
sample = 1
s_time = time.time()
Sh = np.zeros(6)
for s in range(sample):
    mu_s = np.random.normal(1, 0, size=(T, num_s))
    mu_a = np.random.normal(1, 0, size=(T, num_a))
    # beta的分布可以自己设置
    beta_s = np.random.normal(0.5, 0, size=(T, num_s, num_s))
    beta_a = np.random.normal(0.5, 0, size=(T, num_s, num_a))
    # 接下来抽取e的协方差矩阵，依然从均匀分布中抽取，假设不同时间的fatcor不相关
    # 时间1的方差
    mean_1 = np.array([0, 0, 0])  # 均值向量
    c_e1 = np.random.uniform(0.1, 0.1, 3)
    v_e1 = np.random.uniform(0.5, 0.5, 3)
    cov_1 = np.array([[v_e1[0], c_e1[0], c_e1[1]],
                      [c_e1[0], v_e1[1], c_e1[2]],
                      [c_e1[1], c_e1[2], v_e1[2]]])  # 协方差矩阵
    # 时间2的方差
    mean_2 = np.array([0, 0, 0])  # 均值向量
    c_e2 = np.random.uniform(-1, -1, 3)
    v_e2 = np.random.uniform(3, 3, 3)
    cov_2 = np.array([[v_e2[0], c_e2[0], c_e2[1]],
                      [c_e2[0], v_e2[1], c_e2[2]],
                      [c_e2[1], c_e2[2], v_e2[2]]])  # 协方差矩阵
    # 两个随机变量之间的相关性是0
    R1 = np.dot(alpha[1],np.eye(num_s)) + np.dot(alpha[2],R(1,1)) + np.dot(alpha[3],R(1,2))
    R2 = np.dot(alpha[2],np.eye(num_s)) + np.dot(alpha[3],R(2,2))
    R3 = np.dot(alpha[3],np.eye(num_s))
    RR = np.concatenate((R1,R2,R3))


    for i in range(5):
        for j in range(i+1,6):
            Shapley_value[i][j] = Shapley_value[i][j] + shapley(i,j)
    for i in range(6):
        Sh[i] = Sh[i] + Sh_single(i)


Shapley_value = np.round(Shapley_value/sample, decimals = 3)
Sh = np.round(Sh/sample, decimals = 3)
e_time = time.time()
#print(e_time-s_time)


def custom_formatter(x):
    return repr(x) + ','


np.set_printoptions(formatter={'all': custom_formatter})

print(Shapley_value)

print(Sh)

def percent_choose(Sh,percent):
    per = 0
    Sh_choose = []
    i = 1
    while per < percent:
        per = per + np.sort(Sh)[-i]/np.sum(Sh)
        Sh_choose.append(np.argsort(Sh)[-i])
        i = i + 1
    return per, Sh_choose

per,Sh_choose = percent_choose(Sh, 0.9)
print(per)
print(Sh_choose)